In [1]:
print("OK")

OK


# The goal

Reply to a customer email
1. get the email
2. categorize this as a "sales", "custom enquiry", "off topic", "customer complaint"
3. use the category to research info needed for the reply
4. take the info and check if there is enough to answer the email
5. write a reply
6. check the reply
7. save the email

### Agents
- categorizer
- researcher (web for now would be internal RAG)
- email writer


### Tasks
- categorize email
- research the answer
- write the email

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.environ["GROQ_API_KEY"]

## Utilities

In [5]:
import json  # Import the JSON module to parse JSON strings
from langchain_core.agents import AgentFinish

agent_finishes  = []

import json
from typing import Union, List, Tuple, Dict
from langchain.schema import AgentFinish

call_number = 0

def print_agent_output(agent_output: Union[str, List[Tuple[Dict, str]], AgentFinish], agent_name: str = 'Generic call'):
    global call_number  # Declare call_number as a global variable
    call_number += 1
    with open("crew_callback_logs.txt", "a") as log_file:
        # Try to parse the output if it is a JSON string
        if isinstance(agent_output, str):
            try:
                agent_output = json.loads(agent_output)  # Attempt to parse the JSON string
            except json.JSONDecodeError:
                pass  # If there's an error, leave agent_output as is

        # Check if the output is a list of tuples as in the first case
        if isinstance(agent_output, list) and all(isinstance(item, tuple) for item in agent_output):
            print(f"-{call_number}----Dict------------------------------------------", file=log_file)
            for action, description in agent_output:
                # Print attributes based on assumed structure
                print(f"Agent Name: {agent_name}", file=log_file)
                print(f"Tool used: {getattr(action, 'tool', 'Unknown')}", file=log_file)
                print(f"Tool input: {getattr(action, 'tool_input', 'Unknown')}", file=log_file)
                print(f"Action log: {getattr(action, 'log', 'Unknown')}", file=log_file)
                print(f"Description: {description}", file=log_file)
                print("--------------------------------------------------", file=log_file)

        # Check if the output is a dictionary as in the second case
        elif isinstance(agent_output, AgentFinish):
            print(f"-{call_number}----AgentFinish---------------------------------------", file=log_file)
            print(f"Agent Name: {agent_name}", file=log_file)
            agent_finishes.append(agent_output)
            # Extracting 'output' and 'log' from the nested 'return_values' if they exist
            output = agent_output.return_values
            # log = agent_output.get('log', 'No log available')
            print(f"AgentFinish Output: {output['output']}", file=log_file)
            # print(f"Log: {log}", file=log_file)
            # print(f"AgentFinish: {agent_output}", file=log_file)
            print("--------------------------------------------------", file=log_file)

        # Handle unexpected formats
        else:
            # If the format is unknown, print out the input directly
            print(f"-{call_number}-Unknown format of agent_output:", file=log_file)
            print(type(agent_output), file=log_file)
            print(agent_output, file=log_file)


In [6]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            # api_key=os.getenv("GROQ_API_KEY"),
            model="llama3-70b-8192"
        )

## Tools

In [7]:
from crewai import Crew, Agent, Task, Process
from langchain_community.tools import DuckDuckGoSearchRun

In [8]:
from datetime import datetime
from random import randint
from langchain.tools import tool

In [9]:
import json
import os

import requests
from langchain.tools import tool

import os
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools

search_tool = DuckDuckGoSearchRun()


In [12]:
search_tool.run("2024 euro cup date")

'When do EURO 2024 tickets go on sale? The latest UEFA EURO 2024 ticketing information can be found here.. EURO 2024 key dates. Final tournament 14 June 2024: UEFA EURO 2024 opening game, Munich ... Dates: 14 June - 14 July: Teams: 24: Venue(s) 10 (in 10 host cities) ← 2020. 2028 → . The 2024 UEFA European Football Championship, commonly referred to as UEFA Euro 2024 (stylised as UEFA EURO 2024) or simply Euro 2024, will be the 17th ... Here are the key dates for the Euro 2024 tournament: ADVERTISEMENT. Group stage: June 14 to June 26. Round of 16: June 29 to July 2. Quarterfinals: July 5 and 6. Semifinals: July 9 and 10. Final ... Find out the dates, venues and kick-off times for Euro 2024, the biggest soccer tournament in Europe. See the knockout bracket and follow your favorite teams on ESPN. Three-time winners Germany will stage the UEFA European Championship in 2024. Live 04/04/2024 EURO 2024: Host cities Get the lowdown on the EURO 2024 host cities and stadiums with our venue gui

c:\Users\arjun.sreekumar\AppData\Local\anaconda3\envs\email_agent\lib\site-packages\curl_cffi\aio.py:204: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


## Agents

In [13]:
class EmailAgents():
    def make_categorizer_agent(self):
        return Agent(
            role='Email Categorizer Agent',
            goal="""take in a email from a human that has emailed out company email address and categorize it \
            into one of the following categories: \
            price_equiry - used when someone is asking for information about pricing \
            customer_complaint - used when someone is complaining about something \
            product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
            customer_feedback - used when someone is giving feedback about a product \
            off_topic when it doesnt relate to any other category """,
            backstory="""You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
            step_callback=lambda x: print_agent_output(x,"Email Categorizer Agent"),
        )

    def make_researcher_agent(self):
        return Agent(
            role='Info Researcher Agent',
            goal="""take in a email from a human that has emailed out company email address and the category \
            that the categorizer agent gave it and decide what information you need to search for for the email writer to reply to \
            the email in a thoughtful and helpful way.
            If you DONT think a search will help just reply 'NO SEARCH NEEDED'
            If you dont find any useful info just reply 'NO USEFUL RESESARCH FOUND'
            otherwise reply with the info you found that is useful for the email writer
            """,
            backstory="""You are a master at understanding what information our email writer needs  to write a reply that \
            will help the customer""",
            llm=GROQ_LLM,
            verbose=True,
            max_iter=5,
            allow_delegation=False,
            memory=True,
            tools=[search_tool],
            step_callback=lambda x: print_agent_output(x,"Info Researcher Agent"),
        )

    def make_email_writer_agent(self):
        return Agent(
            role='Email Writer Agent',
            goal="""take in a email from a human that has emailed out company email address, the category \
            that the categorizer agent gave it and the research from the research agent and \
            write a helpful email in a thoughtful and friendly way.

            If the customer email is 'off_topic' then ask them questions to get more information.
            If the customer email is 'customer_complaint' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'customer_feedback' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'product_enquiry' then try to give them the info the researcher provided in a succinct and friendly way.
            If the customer email is 'price_equiry' then try to give the pricing info they requested.

            You never make up information. that hasn't been provided by the researcher or in the email.
            Always sign off the emails in appropriate manner and from Sarah the Resident Manager.
            """,
            backstory="""You are a master at synthesizing a variety of information and writing a helpful email \
            that will address the customer's issues and provide them with helpful information""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
            step_callback=lambda x: print_agent_output(x,"Email Writer Agent"),
        )



## Tasks

In [14]:
class EmailTasks():
    # Define your tasks with descriptions and expected outputs
    def categorize_email(self, email_content):
        return Task(
            description=f"""Conduct a comprehensive analysis of the email provided and categorize into \
            one of the following categories:
            price_equiry - used when someone is asking for information about pricing \
            customer_complaint - used when someone is complaining about something \
            product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
            customer_feedback - used when someone is giving feedback about a product \
            off_topic when it doesnt relate to any other category \

            EMAIL CONTENT:\n\n {email_content} \n\n
            Output a single cetgory only""",
            expected_output="""A single categtory for the type of email from the types ('price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic') \
            eg:
            'price_enquiry' \
            """,
            output_file=f"email_category.txt",
            agent=categorizer_agent
            )

    def research_info_for_email(self, email_content):
        return Task(
            description=f"""Conduct a comprehensive analysis of the email provided and the category \
            provided and search the web to find info needed to respond to the email

            EMAIL CONTENT:\n\n {email_content} \n\n
            Only provide the info needed DONT try to write the email""",
            expected_output="""A set of bullet points of useful info for the email writer \
            or clear instructions that no useful material was found.""",
            context = [categorize_email],
            output_file=f"research_info.txt",
            agent=researcher_agent
            )

    def draft_email(self, email_content):
        return Task(
            description=f"""Conduct a comprehensive analysis of the email provided, the category provided\
            and the info provided from the research specialist to write an email. \

            Write a simple, polite and too the point email which will respond to the customer's email. \
            If useful use the info provided from the research specialist in the email. \

            If no useful info was provided from the research specialist the answer politely but don't make up info. \

            EMAIL CONTENT:\n\n {email_content} \n\n
            Output a single cetgory only""",
            expected_output="""A well crafted email for the customer that addresses their issues and concerns""",
            context = [categorize_email, research_info_for_email],
            agent=email_writer_agent,
            output_file=f"draft_email.txt",
            )

## Instantiate Agents and Tasks

In [15]:
# EMAIL = """HI there, \n
# I am emailing to find out the current price of Bitcoin. \n

# Can you please help me/

# Thanks,
# John
# """

EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

EMAIL = """HI there, \n
I am emailing to say that the resort weather was way to cloudy and overcast. \n
I wanted to write a song called 'Here comes the sun but it never came'

What should be the weather in Arizona in April?

I really hope you fix this next time.

Thanks,
George
"""

EMAIL = """HI there, \n
Why can't I get to sing?

Thanks,
Ringo
"""

In [16]:
##
agents = EmailAgents()
tasks = EmailTasks()

## Agents
categorizer_agent = agents.make_categorizer_agent()
researcher_agent = agents.make_researcher_agent()
email_writer_agent = agents.make_email_writer_agent()

## Tasks
categorize_email = tasks.categorize_email(EMAIL)
research_info_for_email = tasks.research_info_for_email(EMAIL)
draft_email = tasks.draft_email(EMAIL)


## Crew

crew.ai, one of the emerging frameworks for building multi-agent application. Crew.ai provides a framework for building agents to work together seamlessly, tackling complex tasks with a level of sophistication that mirrors the dynamics of a well-oiled team. Crew.AI is designed to enable AI agents to assume roles, share goals, and operate in a cohesive unit.

### Architecture
The architecture of CrewAI is modular, comprising several key components working together to achieve a well-orchestrated multi-agent system. The following picture shows the key components that provide a framework to build multi-agent LLM applications.

![Image Title](assets\crew.jpg)


Let's go through this picture bottom up so that we understand how they come together.

Tool: A tool object is a utility/equipment that the agents use to perform a specific task efficiently. For example, searching the web, loading documents & reading them, etc. Crewai is built with LangChain, we can use any existing ones from LangChain or we can write our custom tools.

Task: Task, as the name suggests, is the specific task that needs to be executed. An Agent performs a task. We provide various tools that are required to execute a task.

Agents: An agent is like a team member in the crew, with a specific role, background story, goal, and memory. The agent is the core performer of the tasks assigned within the framework. Each CrewAI Agent is a LangChain Agent, but enhanced with a ReActSingleInputOutputParser. This parser is specially modified to better support role-playing, incorporates a binding stop word for contextual focus, and integrates a memory mechanism, using ConversationSummaryMemory for maintaining the context.

Crew: Think of the crew as a team of agents, working together to achieve a particular goal. These agents have a clearly defined way to collaborate to achieve the set of tasks at hand

Process: Process object is the workflow or strategy the crew follows to complete tasks. 3 strategies are defined by the framework

Sequential: This strategy executes the given “tasks” in sequence, and in a particular defined order. This strategy is ideal for any pipeline type of work, where each agent performs a specific task and passes it to the next agent. We will use this strategy in our example today to write a blog, for a given topic.
Hierarchical: This strategy organizes tasks in a hierarchy, and these tasks are delegated in a hierarchical fashion and executed based on chain of command. This is remarkably close to an orchestrator kind of pattern. This is like a manager assigning work to various agents, and validating the results, before completing the work. When we use this strategy, we need to configure a manager_llm, that helps take the right decision. We will be building a solution using this strategy in the next blog.
Consensual Process (Planned): This is one of the most popular strategies, which is yet to be released, where we have agents talking to each other to get the work done. This is based on collaborative decision-making and the decisions are taken democratically. This is not yet released, but we have other multi-system frameworks, and have already implemented this strategy. We will explore this in future blogs

In [17]:
from crewai import Crew, Process

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[categorizer_agent, researcher_agent, email_writer_agent],
    tasks=[categorize_email, research_info_for_email, draft_email],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    step_callback=lambda x: print_agent_output(x,"MasterCrew Agent")
)

In [18]:
# Kick off the crew's work
results = crew.kickoff()

# Print the results
print("Crew Work Results:")
print(results)

# print(f"Categorize Email: {categorize_email.output}")
print(crew.usage_metrics)

 [DEBUG]: == Working Agent: Email Categorizer Agent
 [INFO]: == Starting Task: Conduct a comprehensive analysis of the email provided and categorize into             one of the following categories:
            price_equiry - used when someone is asking for information about pricing             customer_complaint - used when someone is complaining about something             product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \
            customer_feedback - used when someone is giving feedback about a product             off_topic when it doesnt relate to any other category 
            EMAIL CONTENT:

 HI there, 

Why can't I get to sing?

Thanks,
Ringo
 


            Output a single cetgory only


> Entering new CrewAgentExecutor chain...
Thought: After carefully analyzing the email content, I understand that the sender, Ringo, is expressing frustration about not being able to sing, but the email does not 

c:\Users\arjun.sreekumar\AppData\Local\anaconda3\envs\email_agent\lib\site-packages\curl_cffi\aio.py:204: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


I now can give a great answer

Final Answer:

Dear Ringo,

Thank you for reaching out to us about your concerns with singing. I understand that it can be frustrating when you're having trouble with pitch accuracy. 

The good news is that with practice and exercises, most people can learn to sing. Innate abilities can be developed and improved with practice and patience. You might be surprised to know that certain exercises like pole dancing and yoga can even help strengthen your core muscles, which can improve your vocal power.

Additionally, gargling warm salt water can help relieve inflammation and keep your throat moist. It's also important to note that congenital amusia, a disorder that affects 1 in every 75 people, can prevent people from hearing differences in pitch, tone, and rhythm. However, with practice, most people can learn to sing, regardless of innate talent.

If you're interested in improving your singing, I would recommend exploring exercises and practices that can help

In [20]:
# Open and read the contents of the file
with open('draft_email.txt', 'r') as file:
    content = file.read()

# Print the contents of the file
print(content)


Dear Ringo,

Thank you for reaching out to us about your concerns with singing. I understand that it can be frustrating when you're having trouble with pitch accuracy. 

The good news is that with practice and exercises, most people can learn to sing. Innate abilities can be developed and improved with practice and patience. You might be surprised to know that certain exercises like pole dancing and yoga can even help strengthen your core muscles, which can improve your vocal power.

Additionally, gargling warm salt water can help relieve inflammation and keep your throat moist. It's also important to note that congenital amusia, a disorder that affects 1 in every 75 people, can prevent people from hearing differences in pitch, tone, and rhythm. However, with practice, most people can learn to sing, regardless of innate talent.

If you're interested in improving your singing, I would recommend exploring exercises and practices that can help you develop your skills. If you have any more

In [21]:
# Open and read the contents of the file
with open('email_category.txt', 'r') as file:
    content = file.read()

# Print the contents of the file
print(content)

off_topic


In [22]:
# Open and read the contents of the file
with open('research_info.txt', 'r') as file:
    content = file.read()

# Print the contents of the file
print(content)

* Some people may struggle with singing due to pitch accuracy issues, but with practice and exercises, most people can learn to sing.
* Innate abilities can be developed and improved with practice and patience.
* Certain exercises like pole dancing and yoga can help strengthen core muscles, which can improve vocal power.
* Gargling warm salt water can help relieve inflammation and keep the throat moist.
* Congenital amusia is a disorder that affects 1 in every 75 people, preventing them from hearing differences in pitch, tone, and rhythm.
* With practice, most people can learn to sing, regardless of innate talent.
